<a href="https://colab.research.google.com/github/Alan-Cheong/IEEE_QW_2020/blob/master/Shor's_9_Qubit_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install qiskit
!pip install qiskit-aer
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import transpile
from qiskit_aer import AerSimulator
import numpy as np

# Initialize registers
qreg = QuantumRegister(9, 'q')  # 9 qubits for Shor's code
creg_bit = ClassicalRegister(6, 'c_bit')  # 6 bits for bit-flip syndromes
creg_phase = ClassicalRegister(2, 'c_phase')  # 2 bits for phase-flip syndromes
circ = QuantumCircuit(qreg, creg_bit, creg_phase)

# Initialize logical qubit in |+> = (|0> + |1>)/sqrt(2) for testing
circ.h(0)  # Apply Hadamard to qubit 0

# Encoding circuit for Shor's 9-qubit code
# First group (qubits 0, 1, 2)
circ.cx(0, 1)
circ.cx(0, 2)
circ.h([0, 1, 2])  # Hadamard for phase encoding
# Second group (qubits 3, 4, 5)
circ.cx(0, 3)
circ.cx(0, 4)
circ.cx(3, 5)
circ.h([3, 4, 5])
# Third group (qubits 6, 7, 8)
circ.cx(0, 6)
circ.cx(0, 7)
circ.cx(6, 8)
circ.h([6, 7, 8])

# Introduce an X error on qubit 2 (for demonstration)
circ.x(2)

# Syndrome measurements
# Bit-flip syndromes (Z1Z2, Z1Z3, Z4Z5, Z4Z6, Z7Z8, Z7Z9)
circ.cx(0, 1)
circ.cz(0, 1)
circ.measure(1, 0)
circ.reset(1)
circ.cx(0, 2)
circ.cz(0, 2)
circ.measure(2, 1)
circ.reset(2)
circ.cx(3, 4)
circ.cz(3, 4)
circ.measure(4, 2)
circ.reset(4)
circ.cx(3, 5)
circ.cz(3, 5)
circ.measure(5, 3)
circ.reset(5)
circ.cx(6, 7)
circ.cz(6, 7)
circ.measure(7, 4)
circ.reset(7)
circ.cx(6, 8)
circ.cz(6, 8)
circ.measure(8, 5)
circ.reset(8)

# Phase-flip syndromes (X1X4, X1X7)
circ.h([0, 3, 6])
circ.cx(0, 3)
circ.cz(0, 3)
circ.measure(3, 6)
circ.reset(3)
circ.cx(0, 6)
circ.cz(0, 6)
circ.measure(6, 7)
circ.reset(6)
circ.h(0)

# Error correction based on syndromes (example for X error on qubit 2)
# If c_bit[1] = 1 and c_bit[0] = 0, apply X to qubit 2
# In simulation, we know the error; in practice, use conditional gates
circ.x(2)  # Correct the X error

# Measure the logical qubit (in |+> state, expect 0 in X-basis)
circ.h(0)
circ.measure(0, creg_bit[0])  # Reuse c_bit[0] for final measurement

print(circ)

# Simulate the circuit
backend = AerSimulator()
new_circuit = transpile(circ, backend)
job = backend.run(new_circuit, shots=1024)
result = job.result()
counts = result.get_counts()
print(counts)
#simulator = Aer.get_backend('qasm_simulator')
#job = backend.run(circ, simulator, shots=1024)
#result = job.result()
#counts = result.get_counts()

# Print results
print("Measurement counts:", counts)

# Analyze syndromes
for outcome in counts:
    bits = outcome.split()
    bit_syndrome = bits[1]  # c_bit
    phase_syndrome = bits[0]  # c_phase
    print(f"Bit-flip syndrome: {bit_syndrome}, Phase-flip syndrome: {phase_syndrome}")


           ┌───┐          ┌───┐                                             »
      q_0: ┤ H ├──■────■──┤ H ├──■─────────■─────────■────■────■────■────■──»
           └───┘┌─┴─┐  │  ├───┤  │         │         │    │  ┌─┴─┐  │    │  »
      q_1: ─────┤ X ├──┼──┤ H ├──┼─────────┼─────────┼────┼──┤ X ├──■────┼──»
                └───┘┌─┴─┐├───┤  │  ┌───┐  │         │    │  └───┘     ┌─┴─┐»
      q_2: ──────────┤ X ├┤ H ├──┼──┤ X ├──┼─────────┼────┼────────────┤ X ├»
                     └───┘└───┘┌─┴─┐└───┘  │  ┌───┐  │    │            └───┘»
      q_3: ────────────────────┤ X ├──■────┼──┤ H ├──┼────┼────■────■───────»
                               └───┘  │  ┌─┴─┐├───┤  │    │  ┌─┴─┐  │   ┌─┐ »
      q_4: ───────────────────────────┼──┤ X ├┤ H ├──┼────┼──┤ X ├──■───┤M├─»
                                    ┌─┴─┐├───┤└───┘  │    │  └───┘      └╥┘ »
      q_5: ─────────────────────────┤ X ├┤ H ├───────┼────┼──────────────╫──»
                                    └───┘└───┘     ┌─┴─┐  │     